In [ ]:
import pandas as pd
import sqlite3 as s3
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

go_sales = s3.connect('go_sales.sqlite')

order_details = pd.read_sql_query('SELECT * FROM order_details', go_sales)
order_header = pd.read_sql_query('SELECT * FROM order_header', go_sales)

In [ ]:
merged = pd.merge(order_details, order_header, left_on='ORDER_NUMBER', how='inner', right_on='ORDER_NUMBER')
df = merged.loc[:, ['PRODUCT_NUMBER', 'QUANTITY', 'ORDER_DATE']]

df['ORDER_DATE'] = pd.to_datetime(df['ORDER_DATE'])
df['MONTH'] = df['ORDER_DATE'].dt.month
df['YEAR'] = df['ORDER_DATE'].dt.year
df = df.drop('ORDER_DATE', axis=1)
sales_per_month = df.groupby(['YEAR', 'MONTH'])['QUANTITY'].sum().reset_index()
pd.DataFrame(sales_per_month)

In [ ]:
x = sales_per_month.drop('QUANTITY', axis=1)
y = sales_per_month['QUANTITY']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
reg_model = LinearRegression().fit(x_train, y_train)

year_2023 = pd.DataFrame({'YEAR': [2023] * 12, 'MONTH': range(1, 13)})

predicted_quantities_2023 = reg_model.predict(year_2023)
prediction_2023_df = pd.DataFrame({'Year': year_2023['YEAR'], 'Month': year_2023['MONTH'], 'Predicted Sales': predicted_quantities_2023})
prediction_2023_df['Predicted Sales'] = prediction_2023_df['Predicted Sales'].astype(int)

prediction_2023_df

In [ ]:
int(prediction_2023_df['Predicted Sales'].sum())